# Creating ML apps with Gradio

<a href="https://colab.research.google.com/drive/1rghHhR4TShqkYk4qbZvg0N5oAku2USon" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
</a>

Return to the [castle](https://github.com/Nkluge-correa/TeenyTinyCastle).

In this repository, you will find examples of how to create simple interfaces for things like, for example, [sentiment analysis](https://github.com/Nkluge-correa/TeenyTinyCastle/blob/master/ML-Explainability/NLP/model_maker.ipynb), and even a [playground for HuggingFace language models](https://github.com/Nkluge-correa/TeenyTinyCastle/blob/master/ML-Explainability/NLP%20Playgroung/playground.py). These interfaces can be used to create ML applications since all of them use things like `Flask` for the backend, and bootstrap and CSS for the front end part.

However, there are simpler ways to create demo ML apps. And one of the simpler ways is by using `Gradio`.

[Gradio](https://gradio.app/) is a free and open-source Python library that allows you to develop an easy-to-use customizable component demo for your machine learning model that anyone can use anywhere. Gradio integrates with the most popular Python libraries used for ML and Data Science, including [Scikit-learn](https://scikit-learn.org/stable/), [PyTorch](https://pytorch.org/), [NumPy](https://numpy.org/), [seaborn](https://seaborn.pydata.org/), [pandas](https://pandas.pydata.org/), [TensorFlow](https://www.tensorflow.org/), and many others.

<img src="https://pypi-camo.global.ssl.fastly.net/a95ef5913dc4cc84d2155ff690a0fa0d4c33d7e2/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f67726164696f2d6170702f67726164696f2f6d61696e2f726561646d655f66696c65732f67726164696f2e737667" alt="gradio-image" width="400px">

[Source](https://pypi.org/project/gradio/).

Let's first create an application to [recognize digits](https://github.com/Nkluge-correa/TeenyTinyCastle/blob/master/ML-Intro-Course/7_MNIST_numpy.ipynb), one of the first tasks in ML that we explore in our mini course.

For this application, instead of training a dense feed-forward model, we will train a convolutional neural network (`CNN`) since this is the standard in most computer vision applications. You can find other examples of how to build CNNs on [this](https://github.com/Nkluge-correa/TeenyTinyCastle/blob/master/ML-Explainability/CV/CNN_model_maker.ipynb) notebook.

> **Note:** If you do not want to train this model from scratch, you can download it directly from the Hub 🤗 ([AiresPucrs/digit-classifier](https://huggingface.co/AiresPucrs/digit-classifier)).

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

train_images = x_train.reshape(x_train.shape[0], 28, 28, 1)
test_images = x_test.reshape(x_test.shape[0], 28, 28, 1)
val_images = x_val.reshape(x_val.shape[0], 28, 28, 1)

train_labels = tf.keras.utils.to_categorical(y_train)
test_labels = tf.keras.utils.to_categorical(y_test)
val_labels = tf.keras.utils.to_categorical(y_val)

print('Training Set Size: '), print(x_train.shape)
print('Validation Set Size: '), print(x_val.shape)
print('Test Set Size: '), print(x_test.shape)


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(20, (5,5), padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Conv2D(50, (5,5), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])


opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

model.summary()

callbacks = [tf.keras.callbacks.ModelCheckpoint("digit-classifier.h5",
                                                save_best_only=True),
            tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                            patience=3,
                                            verbose=1,
                                            mode="auto",
                                            baseline=None,
                                            restore_best_weights=True)]

history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    epochs=10,
                    batch_size=256,
                    verbose=2,
                    callbacks=callbacks)


test_loss_score, test_acc_score = model.evaluate(test_images, test_labels)

print(f'Final Loss: {round(test_loss_score, 2)}.')
print(f'Final Performance: {round(test_acc_score * 100, 2)} %.')

With our trained and saved model, creating an application with gradio takes not much more than 20 lines of code. You can style certain components with HTML, Markdown, and CSS. Applications created with gradio can even be hosted on the `gradio.app` for 72 hours, free of charge. Just use launch the application with the `share` argument equal to `True` (`demo.launch(share=True)`).

Permanent hosting can be easily done through the [HuggingFace Spaces](https://www.huggingface.co/spaces), or any PaaS (Platforms as a service) you wish to use, like [Heroku](https://heroku.com/) or [Render](https://render.com/).

You can check our Hugging Face space 🤗🏰  with the two applications worked on in this notebook:
[Digit Classifier](https://huggingface.co/spaces/AiresPucrs/play-gradio)
and [Basic Chatbot](https://huggingface.co/spaces/AiresPucrs/Basic-Chatbot).

> Note: First, you need to install `gradio` through the command
pip install. We will also be installing the `huggingface_hub` library to download our trained model.


In [1]:
!pip install gradio==3.35.2 huggingface_hub["tensorflow"] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 2.2 MB/s et

In [ ]:
import gradio as gr
from huggingface_hub import from_pretrained_keras

model = from_pretrained_keras("AiresPucrs/digit-classifier")

classes = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

def predict(img):
  img = img.reshape(1, 28, 28, 1)
  prediction = model.predict(img, verbose=0).tolist()[0]
  return {classes[i]: prediction[i] for i in range(10)}

title = "Digit Classifier - By Teeny-Tiny Castle 🏰"

head = (
  "<center>"
  "<img src='https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png' width=400>"
  "This model was trained to classify numbers (from 0 to 9). To test it, write your number in the space provided."
  "</center>"
)


ref = (
"<center>"
"Return to the <a href='https://github.com/Nkluge-correa/TeenyTinyCastle)'>castle</a>."
"</center>")

# create interface
demo = gr.Interface(fn=predict,
             inputs="sketchpad",
             outputs=gr.Label(num_top_classes=3),
             allow_flagging="never",
             title=title,
             description=head,
             article=ref)

# launch interface
demo.launch()

You can even use Gradio to create applications that are not ML-based. For example, below, we show you how to create a closed-domain chatbot with a small number of answers and questions (_a basic rules-based system that performs n-gram search_). To see the full implementation of this bot, go to [this link](https://github.com/Nkluge-correa/Aira/tree/master/Aira-1).

> Note: You need to install `unidecode` first!

In [3]:
!pip install unidecode -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.0 MB/s eta 0:00:00


In [5]:
from statistics import mode
import urllib.request
import gradio as gr
import unidecode
import requests
import string
import json

import urllib.request

# Download keys_en.json
urllib.request.urlretrieve('https://github.com/Nkluge-correa/Aira/raw/master/Aira-1/data/generated_data/keys_en.json', 'keys_en.json')

# Download answers_en.txt
urllib.request.urlretrieve('https://github.com/Nkluge-correa/Aira/raw/master/Aira-1/data/original_data/answers_en.txt', 'answers_en.txt')

# Load data from files
with open('answers_en.txt', encoding='utf-8') as fp:
    answers = [line.strip() for line in fp]

with open('keys_en.json') as json_file:
    dictionary = json.load(json_file)

def generate_ngrams(text, WordsToCombine):
    """
    Generates n-grams of length WordsToCombine from the input text.

    Args:
        text: A string representing the input text
        WordsToCombine: An integer representing the
            size of the n-grams to be generated

    Returns:
        A list of n-grams generated from the input text, where each
        n-gram is a list of WordsToCombine words
    """
    words = text.split()
    output = []
    for i in range(len(words) - WordsToCombine+1):
        output.append(words[i:i+WordsToCombine])
    return output


def make_keys(text, WordsToCombine):
    """
    Given a text and a number of words to combine, returns
    a list of keys that correspond to all possible combinations
    of n-grams (sequences of n consecutive words) in the text.

    Args:
        - text (str): The input text.
        - WordsToCombine (int): The number of words to combine.

    Returns:
        - sentences (list of str): A list of all the keys, which are
        the n-grams in the text.
    """
    gram = generate_ngrams(text, WordsToCombine)
    sentences = []
    for i in range(0, len(gram)):
        sentence = ' '.join(gram[i])
        sentences.append(sentence)
    return sentences


def chat(message, history):
    """
    A function that generates a response to a user input message
    based on a pre-built dictionary of responses.

    Args:
        message (str): A string representing the user's input message.
        history (list): A list of tuples containing previous
        messages and responses.

    Returns:
        tuple: A tuple containing two lists of tuples. The first list is
        the original history with the user's input message and the bot's
        response appended as a tuple. The second list is an updated history
        with the same tuples.
    """
    history = history or []
    text = message.lower()
    sentences = []
    values = []
    new_text = text.translate(str.maketrans('', '', string.punctuation))
    new_text = unidecode.unidecode(new_text)

    if len(new_text.split()) == 1:
        if new_text in dictionary.keys():
            l = [dictionary[new_text]] * 100
            values.append(l)
        new_text = new_text + ' ' + new_text

    else:
        if new_text in dictionary.keys():
            l = [dictionary[new_text]] * 100
            values.append(l)

    for i in range(1, len(new_text.split()) + 1):
        sentence = make_keys(new_text, i)
        sentences.append(sentence)

    for i in range(len(sentences)):
        attention = sentences[i]
        for i in range(len(attention)):
            if attention[i] in dictionary.keys():
                l = [dictionary[attention[i]]] * i
                values.append(l)

    if len([item for sublist in values for item in sublist]) == 0:
        bot_input_ids = "I'm sorry, either I didn't understand the question, or it is not part of my domain of expertise... :( Try asking it in another way or using other words. Maybe then I can help you!"
        history.append((message, bot_input_ids))
        return history, history

    else:
        values = [item for sublist in values for item in sublist]
        prediction = mode(values)
        bot_input_ids = answers[int(prediction)-1]
        history.append((message, bot_input_ids))
        return history, history

title = "Basic Chatbot - By Teeny-Tiny Castle 🏰"

head = (
  "<center>"
  "<img src='https://d2vrvpw63099lz.cloudfront.net/do-i-need-a-chatbot/header-chat-box.png' width=400>"
  "This is an example of a rules-based closed domain chatbot. It knows a couple of answers to questions related to AI."
  "<br>"
  "</center>"
)

ref = (
"<center>"
"To see its full version (ML style) of this bot, go to <a href='https://playground.airespucrs.org/aira'>this link</a>."
"</center>")

# create a chat interface
chatbot = gr.Chatbot()

demo = gr.Interface(
    chat,
    ["text", "state"],
    [chatbot, "state"],
    allow_flagging="never",
    title=title,
    description=head,
    article=ref
)

demo.launch()

--2023-12-15 15:42:48--  https://github.com/Nkluge-correa/Aira/raw/master/Aira-1/data/generated_data/keys_en.json
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Nkluge-correa/Aira/master/Aira-1/data/generated_data/keys_en.json [following]
--2023-12-15 15:42:48--  https://raw.githubusercontent.com/Nkluge-correa/Aira/master/Aira-1/data/generated_data/keys_en.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 568709 (555K) [text/plain]
Saving to: ‘keys_en.json.1’

keys_en.json.1      100%[===================>] 555.38K  --.-KB/s    in 0.04s   

2023-12-15 15:42:48 (13.3 MB/s) - ‘keys_en.json.1’ s

<IPython.core.display.Javascript object>

Now you know how to create simple AI applications to show and share with your friends and colleagues!


To see its full version (ML style) of this bot, go to <a href='https://nkluge-correa.github.io/Aira/'>this link</a>. The implementation of this gradio app can be found [on the Hub](https://huggingface.co/spaces/nicholasKluge/Aira-Demo)!🤗

---

Return to the [castle](https://github.com/Nkluge-correa/TeenyTinyCastle).